In [1]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import praw
from psaw import PushshiftAPI

api = PushshiftAPI()

import datetime as dt

start_epoch=int(dt.datetime(2021,7,20).timestamp())

list1=list(api.search_submissions(after=start_epoch,
                            subreddit='worldnews',
                            filter=['title']))

In [3]:
new1=[None]*len(list1)

In [4]:
for i in range(0,len(list1)):
    new1[i]=list1[i][3]['title']

s=new1
new=s

In [5]:
for i in range(0,len(s)):
    txt=s[i]
    s[i]=s[i].replace('%','percent')

In [6]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize


for i in range(0,len(s)):
    text = s[i]
    text_tokens = word_tokenize(text)

    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    new[i]=' '.join(tokens_without_sw)

import nltk
nltk.download('averaged_perceptron_tagger')
for i in range(0,len(s)):   
    sentence = new[i]
    tagged_sentence = nltk.tag.pos_tag(sentence.split())
    edited_sentence = [word for word,tag in tagged_sentence if tag != 'JJ' and tag != 'JJR' and tag != 'JJS' and tag != 'RB' and tag != 'RBR' and tag != 'RBS']
    new1[i]=' '.join(edited_sentence)
    
   

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tristanmantle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tristanmantle/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [21]:

new1=list(filter(lambda a: a != '', new1))

In [40]:
 

import nltk
from pycorenlp import *
import collections

l = [None] * (len(new1))

for i in range(0,(len(new1))):
    nlp=StanfordCoreNLP('http://localhost:9000/')
    x=new1[i]
    output = nlp.annotate(x, properties={"annotators":"tokenize,ssplit,pos,depparse,natlog,openie","triple.strict":"true",
                                     "outputFormat": "json",
                                        "openie.max_entailments_per_clause":"1"})
    
    result = [output["sentences"][0]["openie"] for item in output]
    if len(result[0])>0:
        z=len(result[0])
        relationSent=result[0][z-1]['relation'],result[0][z-1]['subject'],result[0][z-1]['object']
        l[i]=list(relationSent)
    else:
        l[i]=[None,None,None]
       

In [97]:
l.reverse()  
df = pd.DataFrame(l, columns =['P', 'S', 'O'], dtype = float)

In [98]:
df1 = df.replace(to_replace='None', value=np.nan).dropna()
df1.shape[0]/df.shape[0]



0.4663002603413364

In [99]:
triples=df1.values

array([['stop', 'Jerry', 'sales territories'],
       ['jumps', 'gymnast', 'Olympic Village bed news'],
       ['tossed', "Peru 's authority",
        'leaving Pedro Castillo doorstep presidency'],
       ...,
       ['killed', 'family seven', 'Turkey'],
       ['killed', 'Iran attack tanker Oman', 'two'],
       ['killed', 'family seven', 'Turkey']], dtype=object)

In [100]:
import math
train=df1[:math.floor(0.75*len(df1))]
valid=df1[math.floor(0.75*len(df1)):math.floor(0.90*len(df1))]
test=df1[math.floor(0.9*len(df1)):]

In [101]:
train_triples=train.values
valid_triples=valid.values
test_triples=test.values

In [109]:
all_data=df1.values

,P,S,O
1,stop,Jerry,sales territories
2,jumps,gymnast,Olympic Village bed news
5,tossed,Peru 's authority,leaving Pedro Castillo doorstep presidency
7,Reviews For,Reviews,Best Product
8,visit India at_time,Afghan Army Chief,week
...,...,...,...
6908,offer,sanctions Syria faction,solace victims
6910,meets,Cuba sanctions Biden,Cuban leaders
6911,killed,family seven,Turkey
6912,killed,Iran attack tanker Oman,two


In [118]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import urllib.parse

urls=[]

for z in range(0,len(train_triples)):
    try:
        ## initial consts
        BASE_URL = 'http://api.dbpedia-spotlight.org/en/annotate?text={text}&confidence={confidence}&support={support}'
        TEXT = train_triples[z][2]
        CONFIDENCE = '0.9'
        SUPPORT = '10'
        REQUEST = BASE_URL.format(
            text=urllib.parse.quote_plus(TEXT), 
            confidence=CONFIDENCE, 
            support=SUPPORT
        )
        HEADERS = {'Accept': 'application/json'}
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        all_urls = []

        r = requests.get(url=REQUEST, headers=HEADERS)
        response = r.json()
        resources = response['Resources']
        for res in resources:
            all_urls.append(res['@URI'])
        
        urls = all_urls + urls

    except:
        
        pass # doing nothing on exception
    

In [147]:
urls = list(dict.fromkeys(urls))
test_urls=urls

In [151]:

    columns = ['S','P', 'O']
    data = []
    whole_df= pd.DataFrame(data,columns=columns)
    for i in range(0,len(test_urls)):
                
                line = 'DESCRIBE '
                output_line=line +'<' + test_urls[i] + '>'



                sparql = SPARQLWrapper("http://dbpedia.org/sparql")

                q="""

                    {}

                """.format(output_line)

                sparql.setQuery(q)



                sparql.setReturnFormat(JSON)
                results = sparql.query().convert()
                
                

                s_l = [None] * (len(results['results']['bindings']))
                p_l = [None] * (len(results['results']['bindings']))
                o_l = [None] * (len(results['results']['bindings']))
                for i in range(1,(len(results['results']['bindings']))):

                    s_l[i]=results['results']['bindings'][i]['s']['value']
                    p_l[i]=results['results']['bindings'][i]['p']['value']
                    o_l[i]=results['results']['bindings'][i]['o']['value']
                    
                    

                extra_df = pd.DataFrame(list(zip(s_l, p_l,o_l)),
                               columns =['S', 'P','O'])
                extra_df=extra_df.iloc[1:]
                
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageWikiLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageExternalLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageLength']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageRedirects']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/property/wikiPageUsesTemplate']
                
                
                extra_df=extra_df.loc[extra_df['P'] != "http://dbpedia.org/ontology/country"]
                extra_df=extra_df.loc[extra_df['P'] != "http://dbpedia.org/property/country"]
                
                
                
                frames=[whole_df,extra_df]

                whole_df=pd.concat(frames)

In [160]:
#test=whole_df.groupby('P').count()
#test.sort_values(by=['O'])

In [166]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import urllib.parse

valid_urls=[]

for z in range(0,len(valid_triples)):
    try:
        ## initial consts
        BASE_URL = 'http://api.dbpedia-spotlight.org/en/annotate?text={text}&confidence={confidence}&support={support}'
        TEXT = valid_triples[z][2]
        CONFIDENCE = '0.9'
        SUPPORT = '10'
        REQUEST = BASE_URL.format(
            text=urllib.parse.quote_plus(TEXT), 
            confidence=CONFIDENCE, 
            support=SUPPORT
        )
        HEADERS = {'Accept': 'application/json'}
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        all_urls = []

        r = requests.get(url=REQUEST, headers=HEADERS)
        response = r.json()
        resources = response['Resources']
        for res in resources:
            all_urls.append(res['@URI'])
        
        valid_urls = all_urls + valid_urls

    except:
        
        pass # doing nothing on exception

In [174]:
    valid_urls = list(dict.fromkeys(valid_urls))
    columns = ['S','P', 'O']
    data = []
    valid_df= pd.DataFrame(data,columns=columns)
    for i in range(0,len(valid_urls)):
                
                line = 'DESCRIBE '
                output_line=line +'<' + valid_urls[i] + '>'



                sparql = SPARQLWrapper("http://dbpedia.org/sparql")

                q="""

                    {}

                """.format(output_line)

                sparql.setQuery(q)



                sparql.setReturnFormat(JSON)
                results = sparql.query().convert()
                
                

                s_l = [None] * (len(results['results']['bindings']))
                p_l = [None] * (len(results['results']['bindings']))
                o_l = [None] * (len(results['results']['bindings']))
                for i in range(1,(len(results['results']['bindings']))):

                    s_l[i]=results['results']['bindings'][i]['s']['value']
                    p_l[i]=results['results']['bindings'][i]['p']['value']
                    o_l[i]=results['results']['bindings'][i]['o']['value']
                    
                    

                extra_df = pd.DataFrame(list(zip(s_l, p_l,o_l)),
                               columns =['S', 'P','O'])
                extra_df=extra_df.iloc[1:]
                
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageWikiLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageExternalLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageLength']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageRedirects']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/property/wikiPageUsesTemplate']
                
                
                extra_df=extra_df.loc[extra_df['P'] != "http://dbpedia.org/ontology/country"]
                extra_df=extra_df.loc[extra_df['P'] != "http://dbpedia.org/property/country"]
                
                extra_df['O'] = extra_df['O'].astype(str)
                extra_df=extra_df[extra_df['S'].str.contains("http://dbpedia.org/",na=False)]
                extra_df=extra_df[extra_df['P'].str.contains("http://dbpedia.org/",na=False)]
                extra_df=extra_df[extra_df['O'].str.contains("http://dbpedia.org/",na=False)]
                
                frames=[valid_df,extra_df]

                valid_df=pd.concat(frames)
                

In [266]:
final_df=pd.concat([train, whole_df], sort=False)
valid_final_df=pd.concat([valid, valid_df], sort=False)

train_all_triple=final_df.values
valid_all_triple=valid_final_df.values

In [264]:
import numpy as np
from ampligraph.latent_features import TransE

model = TransE(batches_count=64, seed=0, epochs=10, k=100, eta=20,optimizer='adam', optimizer_params={'lr':0.0001},loss='pairwise', verbose=True)

model.fit(train_all_triple)

Average TransE Loss:   0.033788: 100%|██████████| 10/10 [11:41<00:00, 70.19s/epoch]


In [273]:
import numpy as np
from ampligraph.latent_features import TransE

model = TransE(batches_count=64, seed=0, epochs=3, k=100, eta=20,optimizer='adam', optimizer_params={'lr':0.0001},loss='pairwise', verbose=True)

model.fit(train_all_triple, early_stopping=True, early_stopping_params={(valid_all_triple[0])})

TypeError: unhashable type: 'numpy.ndarray'